# 厚壁容器计算

In [79]:
%reset -f
import numpy as np
from numpy.linalg.linalg import solve 

def f(ri, ro, pi, po, case, v=0.5, E=1*10**9, sy=200e6):
    '''
    这个函数用来计算厚壁容器
    
    ri, ro : 内径, 外径
    pi, po: 内外压力, 压应力输入正值
    case: 判断条件输入, cas1: 三轴应力状态, 输入1
                      cas2: 活塞, 平面应力状态, z轴无应力, 仅有泊松比引起的轴向应变 输入2
                      cas3: 约束, 平面应变状态, z轴无应变, 有应力 输入3
    r: 待求位置半径
    v, E: 泊松比和杨氏模量
    sy: 许用最大应力
    
    注意!!!!!!  case r,v,E不影响 A,B 的计算结果, 如果只需要A,B 不必考虑这三个参数,输入非零值保证不报错即可
    
    返回结果数量级为10^6 是方程中的A,B
    '''
    C = np.array([[1, -1/ri**2],
                  [1, -1/ro**2]])
    b = np.array([[-pi, -po]]).T
    x = solve(C,b)
    A, B = x
    print(f'A, B = {x.T / 1e6} 10^6\n')
    stress_r_ri, stress_h_ri = np.array([[1, -1/ri**2],[1, 1/ri**2]]) @ x # 内半径处的应力值
    stress_r_ro, stress_h_ro = np.array([[1, -1/ro**2],[1, 1/ro**2]]) @ x # 外半径处的应力值
    print(f'内径处 径向与环向应力分别为{stress_r_ri / 1e6}MPa, {stress_h_ri / 1e6}MPa')
    print(f'外径处 径向与环向应力分别为{stress_r_ro / 1e6}MPa, {stress_h_ro / 1e6}MPa\n')
    k = ro / ri
    if case == 1:
        stress_z = (pi - po * k**2) / (k**2 - 1) 
        strain_z = (stress_z - 2 * A * v) / E
        stress_ys = np.sqrt(((2 * B / ri**2)**2 + (stress_r_ri - stress_z)**2 + (stress_z - stress_h_ri)**2) / 2 )
    elif case == 2:
        stress_z = 0
        strain_z = -v * 2 * A / E
        stress_ys = np.sqrt(((2 * B / ri**2)**2 + (stress_r_ri - stress_z)**2 + (stress_z - stress_h_ri)**2) / 2 )
    elif case == 3:
        stress_z = 2 * A * v
        strain_z = 0
        stress_ys = np.sqrt(((2 * B / ri**2)**2 + (stress_r_ri - stress_z)**2 + (stress_z - stress_h_ri)**2) / 2 )
    print(f'z轴应力 = {stress_z/1e6}MPa, 微应变 = {strain_z*10**6}, 如果没有指定可选参数值则忽略,这两个值没有意义\n')
    # Tresca Yield Criteria
    print(f'Tresca [stress] = {2 * B / ri**2 / 1e6}MPa, 内外最大压差 [pi - po] = {(k**2 - 1) * sy / (2 * k**2) / 1e6}MPa\n')
    # Von Mises Yield Criteria
    if case == 1:
        pi_c = (k**2 - 1) * sy / (k**2 * np.sqrt(3))
    elif case == 2:
        pi_c = (k**2 - 1) * sy / np.sqrt(3 * k**4 + 1)
    elif case == 3:
        pi_c = (k**2 - 1) * sy / np.sqrt(3 * k**4 - 4 * v + 4 * v**2 + 1)
    print(f"""Von Mises [stress] = {stress_ys / 1e6}MPa,
    case1 情况下最大 [pi - po] = {pi_c / 1e6}MPa,
    否则只有当外压为零时 最大许用 [pi] = {pi_c / 1e6}MPa""")
    return x

In [81]:
A, B = f(ri=, ro=, pi=, po=, case=1, v=0.28, E=200e9, sy=650e6/3)

SyntaxError: expected argument value expression (3135927162.py, line 1)

In [83]:
C, D = f(ri=, ro=, pi=, po=, case=1, v=0.28, E=200e9, sy=650e6/3)

SyntaxError: expected argument value expression (1555051600.py, line 1)

In [85]:
E, F = f(ri=, ro=, pi=, po=, case=1, v=0.28, E=200e9, sy=650e6/3)

SyntaxError: expected argument value expression (2005285722.py, line 1)

In [87]:
import matplotlib.pyplot as plt
%matplotlib inline
r1 = np.arange(0.06, 0.08 + 1e-4, 1e-4)
r2 = np.arange(0.08, 0.1 + 1e-4, 1e-4)
y1 = A + B  / r1**2
y2 = C + D / r2**2

plt.figure(figsize=(15,5))  
ax1=plt.subplot(1, 2, 1) # the 3rd number specifies which of the subplots to plot
plt.plot(r1,y1,'ko-')
plt.xticks(fontsize=10)  
plt.yticks(fontsize=10)
plt.grid()

ax2=plt.subplot(1, 2, 2)
plt.plot(r2,y2,'ko-')
plt.xticks(fontsize=10)  # x 轴刻度标签字体大小
plt.yticks(fontsize=10)  # y 轴刻度标签字体大小
plt.grid()



NameError: name 'A' is not defined

In [89]:
r3 = np.arange(0.06, 0.1 + 1e-4, 1e-4)
y3 = E + F/r3**2
plt.figure(figsize=(15,5))  
plt.plot(r3,y3,'ko-')
plt.xticks(fontsize=10)  
plt.yticks(fontsize=10)
plt.grid()

NameError: name 'E' is not defined

In [91]:
y3 = A + E + (B + F) / r1**2
y4 = C + E + (D + F) / r2**2

plt.figure(figsize=(15,5))  
ax1=plt.subplot(1, 2, 1) # the 3rd number specifies which of the subplots to plot
plt.plot(r1,y3,'ko-')
plt.xticks(fontsize=10)  
plt.yticks(fontsize=10)
plt.grid()

ax2=plt.subplot(1, 2, 2)
plt.plot(r2,y4,'ko-')
plt.xticks(fontsize=10)  # x 轴刻度标签字体大小
plt.yticks(fontsize=10)  # y 轴刻度标签字体大小
plt.grid()
print(y1/1e6)
print(y2/1e6)


NameError: name 'A' is not defined